In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', None)

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%load_ext kedro.extras.extensions.ipython
# %reload_kedro

[09/14/23 14:16:03] WARNING  c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\sit ]8;id=787058;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\warnings.py\warnings.py]8;;\:]8;id=749878;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\warnings.py#109\109]8;;\
                             e-packages\kedro\extras\extensions\ipython.py:18: DeprecationWarning:                 
                             kedro.extras.extensions.ipython should be accessed only using the                     
                             alias kedro.ipython. The unaliased name will be removed in Kedro                      
                             0.19.0.                                                                               
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Resolved project path as: c:\Users\syafiq\OneDrive - Malaysian Rating  ]8;id=688509;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=900123;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\
                             Corporation Berhad\MARC Data\Kedro\Razer\razer-settlement.                            
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/14/23 14:16:06] INFO     Kedro project razer_settlement                                         ]8;id=269339;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=383856;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=875897;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=260834;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py#104\104]8;;\
                             'pipelines'                                                                           

[09/14/23 14:16:07] INFO     Registered line magic 'run_viz'                                        ]8;id=993583;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=66765;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\ipython\__init__.py#110\110]8;;\

In [4]:
catalog.list()

['razer_raw',
 'razer_preprocessed',
 'razer_calculated',
 'parameters',
 'params:variable',
 'params:variable.credit',
 'params:variable.boost',
 'params:variable.wechatpaymy',
 'params:variable.grabpay',
 'params:variable.mb2u_qrpay_push',
 'params:variable.tng_ewallet',
 'params:variable.shopeepay',
 'params:variable.rpp_duitnowqr',
 'params:variable.alipay',
 'params:variable.alipayplus',
 'params:variable.unionpay',
 'params:variable.fpx',
 'params:variable.fpx_b2b',
 'params:variable.atome',
 'params:fixed',
 'params:fixed.fpx_mb2u',
 'params:fixed.mb2u',
 'params:fixed.fpx_cimbclicks',
 'params:fixed.cimb_clicks',
 'params:fixed.fpx_rhb',
 'params:fixed.rhb_onl',
 'params:fixed.fpx_pbb',
 'params:fixed.pbebank',
 'params:fixed.fpx_hlb',
 'params:fixed.hlb_onl',
 'params:fixed.fpx_bimb',
 'params:fixed.fpx_amb',
 'params:fixed.amb_w2w',
 'params:fixed.fpx_abmb',
 'params:fixed.alb_onl',
 'params:fixed.fpx_abb',
 'params:fixed.affin_epg',
 'params:fixed.fpx_bmmb',
 'params:fixed.f

In [5]:
# catalog.load('parameters')
params = catalog.load('parameters')

[09/14/23 14:16:58] INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=293874;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=736823;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [6]:
params

{'variable': {'credit': 0.0135,
  'boost': 0.013,
  'wechatpaymy': 0.01,
  'grabpay': 0.013,
  'mb2u_qrpay_push': 0.01,
  'tng_ewallet': 0.013,
  'shopeepay': 0.013,
  'rpp_duitnowqr': 0.0095,
  'alipay': 0.013,
  'alipayplus': 0.013,
  'unionpay': 0.023,
  'fpx': 0.012,
  'fpx_b2b': 0.012,
  'atome': 0.05},
 'fixed': {'fpx_mb2u': 1,
  'mb2u': 1,
  'fpx_cimbclicks': 1,
  'cimb_clicks': 1,
  'fpx_rhb': 1,
  'rhb_onl': 1,
  'fpx_pbb': 1,
  'pbebank': 1,
  'fpx_hlb': 1,
  'hlb_onl': 1,
  'fpx_bimb': 1,
  'fpx_amb': 1,
  'amb_w2w': 1,
  'fpx_abmb': 1,
  'alb_onl': 1,
  'fpx_abb': 1,
  'affin_epg': 1,
  'fpx_bmmb': 1,
  'fpx_bkrm': 1,
  'fpx_bsn': 1,
  'fpx_ocbc': 1,
  'fpx_uob': 1,
  'fpx_hsbc': 1,
  'fpx_scb': 1,
  'fpx_kfh': 1,
  'fpx_b2b_amb': 1.4,
  'fpx_b2b_hlb': 1.4,
  'fpx_b2b_uob': 1.4,
  'fpx_b2b_abb': 1.4,
  'fpx_b2b_hsbc': 1.4,
  'fpx_m2e': 1.4,
  'fpx_b2b_cimb': 1.4,
  'fpx_b2b_bimb': 1.4,
  'fpx_b2b_rhb': 1.4,
  'fpx_b2b_pbb': 1.4,
  'fpx_b2b_kfh': 1.4,
  'fpx_b2b_deutsche': 1

In [5]:
# # params.keys()
# credit_value = None
# for item in params['variable_channels']:
#     if 'credit' in item:
#         credit_value = item['credit']
#         break
        
# print(credit_value)

In [15]:
data_raw = catalog.load('razer_raw')

[09/14/23 16:29:17] INFO     Loading data from 'razer_raw' (ExcelDataSet)...                    ]8;id=84658;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=348494;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [16]:
data_raw.head()

,Order Id,Txn Id,Amount,Status,Type,Provider,Channel,Payment Date
0,R2ERHXK9B4,ICFT27D285,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"11 Sep 2023, 8:59:45 pm"
1,R2ERHXK9B4,UMFHRYMEF6,MYR 15.00,PENDING,RAZERPAY,RAZERPAY,NaN,"11 Sep 2023, 8:58:32 pm"
2,MA5SLY70SV,6569X5ZV1I,MYR 30.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"8 Sep 2023, 11:40:50 am"
3,H32RUF5V3A,0E6VR9GFZT,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"8 Sep 2023, 11:12:22 am"
4,H32RUF5V3A,095QDH96T8,MYR 15.00,PENDING,RAZERPAY,RAZERPAY,NaN,"8 Sep 2023, 11:08:01 am"


In [17]:
data_raw.dtypes

Order Id        object
Txn Id          object
Amount          object
Status          object
Type            object
Provider        object
Channel         object
Payment Date    object
dtype: object

In [ ]:
# # Import date class from datetime module
# from datetime import date
 
# # Returns the current local date
# today = date.today()
# print("Today date is: ", today)

In [18]:
data_success = data_raw[data_raw['Status'] == 'SUCCESS']

In [19]:
# Define a function to transform column names
def transform_column_name(col):
    # Convert capital letter to small letter
    col = col.lower()
    # Replace space with underscore
    col = re.sub(' ', '_', col)
    # Remove leading and trailing special characters
    col = re.sub('^[^a-zA-Z0-9]*|[^a-zA-Z0-9]*$', '', col)
    # Replace special characters with underscore
    col = re.sub('[^a-zA-Z0-9]+', '_', col)
    return col

In [20]:
# Apply the function to the DataFrame columns and rename them
new_columns = {old_col: transform_column_name(old_col) for old_col in data_success.columns}
data_success = data_success.rename(columns=new_columns)

In [21]:
data_success.head()

,order_id,txn_id,amount,status,type,provider,channel,payment_date
0,R2ERHXK9B4,ICFT27D285,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"11 Sep 2023, 8:59:45 pm"
2,MA5SLY70SV,6569X5ZV1I,MYR 30.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"8 Sep 2023, 11:40:50 am"
3,H32RUF5V3A,0E6VR9GFZT,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"8 Sep 2023, 11:12:22 am"
9,9YP3YYMUR3,9S3M6MI7NA,MYR 38.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"5 Sep 2023, 5:15:06 pm"
10,2UZXJGB7U0,0PPZ9KZ2B9,MYR 38.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,"5 Sep 2023, 5:07:26 pm"


In [26]:
data_success.dtypes

order_id                object
txn_id                  object
amount                  object
status                  object
type                    object
provider                object
channel                 object
payment_date    datetime64[ns]
currency                object
txn_amount             float64
channel_copy            object
dtype: object

In [23]:
# Convert the 'payment_date' column to date type
data_success['payment_date'] = pd.to_datetime(data_success['payment_date'])

# Remove the time portion while maintaining the datetime type
data_success['payment_date'] = data_success['payment_date'].dt.floor('D')

# Split the 'amount' column into two columns
data_success[['currency', 'txn_amount']] = data_success.loc[:, ('amount')].str.split(' ', n=1, expand=True)

# Convert the 'txn_amount' column to float datatype
# replace comma with empty if there is any
data_success['txn_amount'] = data_success['txn_amount'].apply(lambda x: x.replace(',', '')).astype(float)

In [24]:
# Convert 'channel' to lowercase and replace spaces with underscores
data_success['channel_copy'] = data_success['channel'].apply(lambda x: x.lower().replace(' ', '_').replace('-', '_'))

In [25]:
data_success.head()

,order_id,txn_id,amount,status,type,provider,channel,payment_date,currency,txn_amount,channel_copy
0,R2ERHXK9B4,ICFT27D285,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-09-11,MYR,15.0,credit
2,MA5SLY70SV,6569X5ZV1I,MYR 30.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-09-08,MYR,30.0,credit
3,H32RUF5V3A,0E6VR9GFZT,MYR 15.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-09-08,MYR,15.0,credit
9,9YP3YYMUR3,9S3M6MI7NA,MYR 38.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-09-05,MYR,38.0,credit
10,2UZXJGB7U0,0PPZ9KZ2B9,MYR 38.00,SUCCESS,RAZERPAY,RAZERPAY,Credit,2023-09-05,MYR,38.0,credit


In [18]:
# Create a dictionary of lists containing the keys
# channel_dict = {key: [inner_dict_key for inner_dict in value for inner_dict_key in inner_dict.keys()] for key, value in params.items()}

channel_dict = {key: list(value.keys()) for key, value in params.items()}

channel_dict

{'variable': ['credit',
  'boost',
  'wechatpaymy',
  'grabpay',
  'mb2u_qrpay_push',
  'tng_ewallet',
  'shopeepay',
  'rpp_duitnowqr',
  'alipay',
  'alipayplus',
  'unionpay',
  'fpx',
  'fpx_b2b',
  'atome'],
 'fixed': ['fpx_mb2u',
  'mb2u',
  'fpx_cimbclicks',
  'cimb_clicks',
  'fpx_rhb',
  'rhb_onl',
  'fpx_pbb',
  'pbebank',
  'fpx_hlb',
  'hlb_onl',
  'fpx_bimb',
  'fpx_amb',
  'amb_w2w',
  'fpx_abmb',
  'alb_onl',
  'fpx_abb',
  'affin_epg',
  'fpx_bmmb',
  'fpx_bkrm',
  'fpx_bsn',
  'fpx_ocbc',
  'fpx_uob',
  'fpx_hsbc',
  'fpx_scb',
  'fpx_kfh',
  'fpx_b2b_amb',
  'fpx_b2b_hlb',
  'fpx_b2b_uob',
  'fpx_b2b_abb',
  'fpx_b2b_hsbc',
  'fpx_m2e',
  'fpx_b2b_cimb',
  'fpx_b2b_bimb',
  'fpx_b2b_rhb',
  'fpx_b2b_pbb',
  'fpx_b2b_kfh',
  'fpx_b2b_deutsche',
  'fpx_b2b_abmb',
  'fpx_b2b_scb',
  'fpx_b2b_ocbc',
  'fpx_b2b_bmmb',
  'fpx_emandate',
  'fpx_directdebit',
  'fpx_agrobank',
  'fpx_b2b_agrobank',
  'fpx_b2b_abbm',
  'fpx_b2b_citibank',
  'fpx_b2b_bkrm',
  'fpx_b2b_pbbe',
  

In [19]:
# Create the new 'rate_type' column based on the 'channel' column
data_success['rate_type'] = data_success['channel_copy'].apply(lambda x: next(key for key, value in channel_dict.items() if x in value))

In [21]:
# Create a new column containing the values based on the keys in the 'channel' column
# data_success['rate'] = data_success['channel_copy'].apply(lambda x: next(d.get(x) for d in params.values() for d in d if x in d))
data_success['rate'] = data_success['channel_copy'].map({k: v for d in params.values() for k, v in d.items()})

In [23]:
# Create the 'txn_charge' column based on the conditions of 'rate_type'
data_success['txn_charge'] = np.where(data_success['rate_type'] == 'fixed', data_success['rate'], 
                                      data_success['txn_amount'] * data_success['rate'])

In [24]:
# Calculate net_amount
data_success['net_amount'] = data_success['txn_amount'] - data_success['txn_charge']

In [ ]:
# data_success.dtypes

In [25]:
# Select on the relevant columns as the final dataset
final_cols = ['payment_date', 'order_id', 'txn_id', 'status', 'type', 'channel',
              'rate_type', 'rate', 'currency', 'txn_amount', 'txn_charge', 
              'net_amount']
razer_final = data_success[final_cols]

In [26]:
razer_final.head(10)

,payment_date,order_id,txn_id,status,type,channel,rate_type,rate,currency,txn_amount,txn_charge,net_amount
0,2023-06-20,D095M1N4QA,95AYFMS1BB,SUCCESS,RAZERPAY,FPX_MB2U,fixed,1.0000,MYR,100.0,1.00,99.00
1,2023-06-20,6H7UWB89UK,HMQ97ZU54X,SUCCESS,RAZERPAY,FPX_AMB,fixed,1.0000,MYR,200.0,1.00,199.00
2,2023-06-20,0C43D647F3,XUDWQ0IKAD,SUCCESS,RAZERPAY,FPX_PBB,fixed,1.0000,MYR,300.0,1.00,299.00
3,2023-06-20,69D31SUP8C,48QFBGA9BZ,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,1000.0,13.50,986.50
4,2023-06-20,V34R5NJLV3,RBDOIGTZND,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,400.0,5.40,394.60
5,2023-06-20,517XZUHNP9,1BQGSXS13I,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,600.0,8.10,591.90
6,2023-06-20,J7LYSHS26E,PNXOLXGQ9O,SUCCESS,RAZERPAY,FPX_MB2U,fixed,1.0000,MYR,100.0,1.00,99.00
7,2023-06-20,WUUWKYV24R,TG74I8ZUXZ,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,500.0,6.75,493.25
10,2023-06-19,9S2KCRBJPK,03JXRBF9I2,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,300.0,4.05,295.95
11,2023-06-19,DNJS4LG3YQ,ZLW69VP0E7,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,100.0,1.35,98.65


In [ ]:
razer_preprocess = catalog.load("razer_preprocessed")

In [ ]:
razer_preprocess.dtypes

In [ ]:
razer_preprocess.head(10)

In [ ]:
interest_date = pd.to_datetime('2023-06-16')

In [ ]:
data_success[data_success['Payment Date'] == interest_date]

In [8]:
razer_calculated = catalog.load("razer_calculated")
razer_calculated.head()

[09/14/23 14:18:58] INFO     Loading data from 'razer_calculated' (ExcelDataSet)...             ]8;id=783292;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=936406;file://c:\Users\syafiq\AppData\Local\anaconda3\envs\kedro-environment\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,payment_date,order_id,txn_id,status,type,channel,rate_type,rate,currency,txn_amount,txn_charge,net_amount
0,"11 Sep 2023, 8:59:45 pm",R2ERHXK9B4,ICFT27D285,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,15,0.2025,14.7975
1,"8 Sep 2023, 11:40:50 am",MA5SLY70SV,6569X5ZV1I,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,30,0.4050,29.5950
2,"8 Sep 2023, 11:12:22 am",H32RUF5V3A,0E6VR9GFZT,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,15,0.2025,14.7975
3,"5 Sep 2023, 5:15:06 pm",9YP3YYMUR3,9S3M6MI7NA,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,38,0.5130,37.4870
4,"5 Sep 2023, 5:07:26 pm",2UZXJGB7U0,0PPZ9KZ2B9,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,38,0.5130,37.4870


In [10]:
# Calculate the sum for the last 3 columns
total_row = razer_calculated.iloc[:, -3:].sum()

# Append the total row to the DataFrame
df = razer_calculated.append(total_row, ignore_index=True)

# Print the updated DataFrame with the total row
df.head()

,payment_date,order_id,txn_id,status,type,channel,rate_type,rate,currency,txn_amount,txn_charge,net_amount
0,"11 Sep 2023, 8:59:45 pm",R2ERHXK9B4,ICFT27D285,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,15.0,0.2025,14.7975
1,"8 Sep 2023, 11:40:50 am",MA5SLY70SV,6569X5ZV1I,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,30.0,0.4050,29.5950
2,"8 Sep 2023, 11:12:22 am",H32RUF5V3A,0E6VR9GFZT,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,15.0,0.2025,14.7975
3,"5 Sep 2023, 5:15:06 pm",9YP3YYMUR3,9S3M6MI7NA,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,38.0,0.5130,37.4870
4,"5 Sep 2023, 5:07:26 pm",2UZXJGB7U0,0PPZ9KZ2B9,SUCCESS,RAZERPAY,Credit,variable,0.0135,MYR,38.0,0.5130,37.4870


In [14]:
# Select columns with float data type
float_columns = df.select_dtypes(include=['float64']).columns

# Format float columns to two decimal points
df[float_columns] = df[float_columns].applymap(lambda x: round(x, 2))

In [13]:
df.tail()

,payment_date,order_id,txn_id,status,type,channel,rate_type,rate,currency,txn_amount,txn_charge,net_amount
145,"3 Jul 2023, 10:10:11 am",HWUQSHM78K,70CTAYNAA1,SUCCESS,RAZERPAY,Credit,variable,0.01,MYR,200.0,2.70,197.30
146,"3 Jul 2023, 10:00:13 am",0X77XULXEK,SSLKJTBDD0,SUCCESS,RAZERPAY,Credit,variable,0.01,MYR,300.0,4.05,295.95
147,"3 Jul 2023, 9:58:29 am",VA9ANKHZJ8,M7V87MDKKS,SUCCESS,RAZERPAY,Credit,variable,0.01,MYR,200.0,2.70,197.30
148,"3 Jul 2023, 9:55:59 am",CM7WSMHUMQ,H0LXJRVT2L,SUCCESS,RAZERPAY,Credit,variable,0.01,MYR,100.0,1.35,98.65
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12761.0,154.62,12606.37
